In [49]:
import os
import io

from google.cloud import vision

In [38]:
os.environ["PROJECT_ID"] = "text-analysis-323506"

In [39]:
%%bash

gcloud iam service-accounts create my-vision-sa --display-name "vision api account"
gcloud iam service-accounts keys create ~/key.json --iam-account my-vision-sa@${PROJECT_ID}.iam.gserviceaccount.com

ERROR: (gcloud.iam.service-accounts.create) Resource in projects [text-analysis-323506] is the subject of a conflict: Service account my-vision-sa already exists within project projects/text-analysis-323506.
- '@type': type.googleapis.com/google.rpc.ResourceInfo
  resourceName: projects/text-analysis-323506/serviceAccounts/my-vision-sa@text-analysis-323506.iam.gserviceaccount.com
created key [0ef42885dee5227562141472bf5a3f4c23d54b2b] of type [json] as [/home/jupyter/key.json] for [my-vision-sa@text-analysis-323506.iam.gserviceaccount.com]


In [40]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.abspath("key.json")

In [42]:
vision_client = vision.ImageAnnotatorClient()

In [52]:
path = 'sign.jpg'

In [53]:
with io.open(path, 'rb') as image_file:
    content = image_file.read()

In [57]:
image_object = vision.Image(content=content)

In [58]:
response = vision_client.text_detection(image=image_object)

In [ ]:
response

In [ ]:
|